In [51]:
# UCI Covertype

In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Add
from tensorflow.keras.optimizers import Adam
import tensorflow as tf


In [53]:
# fetching the Covertype dataset from sklearn
data_bundle = fetch_covtype()
features = data_bundle.data
labels = data_bundle.target

print("Feature matrix shape:", features.shape)
print("Label vector shape:", labels.shape)


Feature matrix shape: (581012, 54)
Label vector shape: (581012,)


In [54]:
# splitting into train, validation, and test
X_temp, X_holdout, y_temp, y_holdout = train_test_split(features, labels, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_holdout, y_holdout, test_size=0.5, random_state=42)
X_train = X_temp
y_train = y_temp



In [55]:
# though there is no missing value in ucl
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_val = imputer.transform(X_val)
X_test = imputer.transform(X_test)

# normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# encode labels and adjust for 0-based indexing
num_classes = 7
y_train_encoded = to_categorical(y_train - 1, num_classes)
y_val_encoded = to_categorical(y_val - 1, num_classes)
y_test_encoded = to_categorical(y_test - 1, num_classes)


In [56]:
input_layer = Input(shape=(X_train_scaled.shape[1],))

# initial dense layers
layer_a = Dense(32, activation='relu')(input_layer)
layer_b = Dense(32, activation='relu')(layer_a)


res_block_output = Add()([layer_a, layer_b])

# deeper layers
layer_c = Dense(32, activation='relu')(res_block_output)
layer_d = Dense(32, activation='relu')(layer_c)

# skip connection from input (with projection)
projected_input = Dense(32)(input_layer)
skip_output = Add()([layer_d, projected_input])

# final layers
layer_e = Dense(32, activation='relu')(skip_output)
output_layer = Dense(num_classes, activation='softmax')(layer_e)

# building model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 54)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_28 (Dense)          │ (None, 32)             │          1,760 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_29 (Dense)          │ (None, 32)             │          1,056 │ dense_28[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_8 (Add)               │ (None, 32)             │              0 │ dense_28[0][0],        │
│                           │                        │                │ dense_29[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_30 (Dense)          │ (None, 32)             │          1,056 │ add_8[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_31 (Dense)          │ (None, 32)             │          1,056 │ dense_30[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_32 (Dense)          │ (None, 32)             │          1,760 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_9 (Add)               │ (None, 32)             │              0 │ dense_31[0][0],        │
│                           │                        │                │ dense_32[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_33 (Dense)          │ (None, 32)             │          1,056 │ add_9[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_34 (Dense)          │ (None, 7)              │            231 │ dense_33[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 7,975 (31.15 KB)

 Trainable params: 7,975 (31.15 KB)

 Non-trainable params: 0 (0.00 B)

In [57]:
model.save("raghav_model.h5")


In [58]:
sample_size = 128
X_small_batch = X_train_scaled[:sample_size]
y_small_batch = y_train_encoded[:sample_size]

# training the model to overfit
history = model.fit(
    X_small_batch, y_small_batch,
    epochs=200,
    batch_size=sample_size,
    verbose=2,
    validation_data=(X_val_scaled, y_val_encoded)
)


Epoch 1/200
1/1 - 5s - 5s/step - accuracy: 0.1562 - loss: 2.1184 - val_accuracy: 0.1676 - val_loss: 2.1761
Epoch 2/200
1/1 - 3s - 3s/step - accuracy: 0.1641 - loss: 2.0364 - val_accuracy: 0.1954 - val_loss: 2.1159
Epoch 3/200
1/1 - 1s - 1s/step - accuracy: 0.1797 - loss: 1.9606 - val_accuracy: 0.2244 - val_loss: 2.0595
Epoch 4/200
1/1 - 4s - 4s/step - accuracy: 0.2266 - loss: 1.8897 - val_accuracy: 0.2550 - val_loss: 2.0063
Epoch 5/200
1/1 - 5s - 5s/step - accuracy: 0.2500 - loss: 1.8228 - val_accuracy: 0.2838 - val_loss: 1.9561
Epoch 6/200
1/1 - 5s - 5s/step - accuracy: 0.2969 - loss: 1.7595 - val_accuracy: 0.3110 - val_loss: 1.9083
Epoch 7/200
1/1 - 5s - 5s/step - accuracy: 0.3203 - loss: 1.6990 - val_accuracy: 0.3366 - val_loss: 1.8627
Epoch 8/200
1/1 - 1s - 1s/step - accuracy: 0.3516 - loss: 1.6412 - val_accuracy: 0.3594 - val_loss: 1.8191
Epoch 9/200
1/1 - 4s - 4s/step - accuracy: 0.3984 - loss: 1.5857 - val_accuracy: 0.3807 - val_loss: 1.7773
Epoch 10/200
1/1 - 5s - 5s/step - acc

In [59]:
train_loss, train_acc = model.evaluate(X_small_batch, y_small_batch, verbose=2)
val_loss, val_acc = model.evaluate(X_val_scaled, y_val_encoded, verbose=2)

print(f"total parameters in model: {model.count_params()}")
print(f"final training loss: {train_loss:.4f}")
print(f"final validation loss: {val_loss:.4f}")

4/4 - 0s - 14ms/step - accuracy: 1.0000 - loss: 0.0196
3632/3632 - 6s - 2ms/step - accuracy: 0.5727 - loss: 3.1683
total parameters in model: 7975
final training loss: 0.0196
final validation loss: 3.1683
